In [ ]:
"""
LALAFO DATASET PROCESSING SCRIPT
================================

Purpose:
--------
Aggregates brand-specific Excel datasets generated by the extraction module,
cleans and normalizes the data, performs statistical imputation,
and exports a final processed dataset.

This module performs:
- Dataset aggregation
- Duplicate removal
- Missing value handling
- Category normalization
- Mileage normalization
- Statistical imputation
- Final export

No data extraction or API communication is performed here.
"""

import pandas as pd
from pandas import ExcelWriter


# ============================================================
# DATASET PATH CONFIGURATION
# ============================================================

ALL_PATH = "../dataset/all_cars/lalafo_results_all_cars.xlsx"

AUDI_PATH = "../dataset/brands/lalafo_results_audi.xlsx"
BMW_PATH = "../dataset/brands/lalafo_results_bmw.xlsx"
HONDA_PATH = "../dataset/brands/lalafo_results_honda.xlsx"
HYUNDAI_PATH = "../dataset/brands/lalafo_results_hyundai.xlsx"
KIA_PATH = "../dataset/brands/lalafo_results_kia.xlsx"
TOYOTA_PATH = "../dataset/brands/lalafo_results_toyota.xlsx"
MERCEDES_PATH = "../dataset/brands/lalafo_results_mercedes.xlsx"
MITSUBISHI_PATH = "../dataset/brands/lalafo_results_mitsubishi.xlsx"
VOLKSWAGEN_PATH = "../dataset/brands/lalafo_results_volkswagen.xlsx"

FILE_PATHS = [
    ALL_PATH,
    AUDI_PATH,
    BMW_PATH,
    HONDA_PATH,
    HYUNDAI_PATH,
    KIA_PATH,
    TOYOTA_PATH,
    MERCEDES_PATH,
    MITSUBISHI_PATH,
    VOLKSWAGEN_PATH,
]


# ============================================================
# DATA AGGREGATION
# ============================================================

dfs = []

# Reading each Excel file into a DataFrame
for path in FILE_PATHS:
    dfs.append(pd.read_excel(path))

# Concatenating all DataFrames into one
df = pd.concat(dfs, ignore_index=True)

# Removing duplicates
df = df.drop_duplicates()


# ============================================================
# MISSING VALUE HANDLING
# ============================================================

df["condition"] = df["condition"].fillna("Used")
df["vin_code"] = df["vin_code"].fillna("Without VIN code")
df["clearance"] = df["clearance"].fillna("Customs Not Cleared")
df["tech_condition"] = df["tech_condition"].fillna("Good condition")


# ============================================================
# CATEGORY NORMALIZATION
# ============================================================

df["tech_condition"] = df["tech_condition"].replace(
    "Good condition, Ideal condition",
    "Good condition"
)

df["fuel_type"] = df["fuel_type"].replace("Gas, Petrol", "Petrol")
df["fuel_type"] = df["fuel_type"].replace("Hybrid, Petrol", "Hybrid")
df["fuel_type"] = df["fuel_type"].replace("Electric, Hybrid, Petrol", "Hybrid")
df["fuel_type"] = df["fuel_type"].replace("Electric, Petrol", "Hybrid")
df["fuel_type"] = df["fuel_type"].replace("Electric, Hybrid", "Hybrid")
df["fuel_type"] = df["fuel_type"].replace("Gas, Hybrid, Petrol", "Hybrid")
df["fuel_type"] = df["fuel_type"].replace("Diesel, Hybrid", "Hybrid")
df["fuel_type"] = df["fuel_type"].replace("Diesel, Petrol", "Petrol")
df["fuel_type"] = df["fuel_type"].replace("Diesel, Gas, Petrol", "Petrol")

df["fuel_type"] = df["fuel_type"].fillna("Petrol")

df["body_type"] = df["body_type"].replace("Van/Minivan", "Van")


# ============================================================
# MILEAGE NORMALIZATION
# ============================================================

# Remove spaces and commas
df["run_km"] = df["run_km"].replace([" ", ","], "", regex=True)

# Convert to float
df["run_km"] = df["run_km"].astype(float)

# Normalize small values (if < 1000 → multiply by 1000)
df["run_km"] = df["run_km"].apply(lambda x: x * 1000 if x < 1000 else x)


# ============================================================
# STATISTICAL IMPUTATION
# ============================================================

# Remove rows without year
df = df.dropna(subset=["year"])

# Calculate mean mileage per year
mean_run_km_by_year = df.groupby("year")["run_km"].mean()


def fill_missing_run_km(row):
    """
    Fill missing mileage using mean mileage for the same production year.
    """
    if pd.isnull(row["run_km"]):
        return mean_run_km_by_year[row["year"]]
    else:
        return row["run_km"]


df["run_km"] = df.apply(fill_missing_run_km, axis=1)


# ============================================================
# EXPORT
# ============================================================

writer = ExcelWriter(
    "../dataset/all_cars/lalafo_results_all_cars_processed.xlsx"
)

df.to_excel(writer, "data")
writer._save()

print("Все сохранено в lalafo_results_processed.xlsx")

In [ ]:
# ============================================================
# SHOWCASE OUTPUTS (For GitHub README / Portfolio)
# ============================================================

import os
import matplotlib.pyplot as plt

SHOWCASE_DIR = "showcase_outputs"
os.makedirs(SHOWCASE_DIR, exist_ok=True)

# 1️⃣ Dataset Preview (first 30 rows)
preview_path = f"{SHOWCASE_DIR}/processed_dataset_preview.csv"
df.head(30).to_csv(preview_path, index=False)

# 2️⃣ Summary Statistics
summary_path = f"{SHOWCASE_DIR}/processed_dataset_summary.csv"
df.describe(include="all").to_csv(summary_path)

# 3️⃣ Price Distribution Chart
if "price" in df.columns:
    plt.figure()
    df["price"].dropna().astype(float).hist(bins=40)
    plt.title("Processed Dataset Price Distribution")
    plt.xlabel("Price (KGS)")
    plt.ylabel("Count")
    plt.tight_layout()

    price_chart_path = f"{SHOWCASE_DIR}/price_distribution.png"
    plt.savefig(price_chart_path)
    plt.close()

# 4️⃣ Average Price by Brand Chart
if "brand" in df.columns and "price" in df.columns:
    avg_price_by_brand = df.groupby("brand")["price"].mean().sort_values()

    plt.figure()
    avg_price_by_brand.plot(kind="bar")
    plt.title("Average Price by Brand")
    plt.xlabel("Brand")
    plt.ylabel("Average Price (KGS)")
    plt.tight_layout()

    brand_chart_path = f"{SHOWCASE_DIR}/average_price_by_brand.png"
    plt.savefig(brand_chart_path)
    plt.close()

print("Showcase files generated in 'showcase_outputs/' directory.")